![gt](https://www.osgeo.org/wp-content/uploads/GeoTrellis.png)

In [ ]:
import $ivy.`org.locationtech.geotrellis::geotrellis-raster:3.5.2`
import $ivy.`org.slf4j:slf4j-simple:1.7.30`

## RasterSource for Landsat
- [RasterSource Overview](https://geotrellis.github.io/geotrellis-workshop/docs/rastersource)
- [RasterSource ScalaDoc](https://geotrellis.github.io/scaladocs/latest/geotrellis/raster/RasterSource.html)

In [ ]:
import geotrellis.raster._
import geotrellis.raster.geotiff.GeoTiffRasterSource

In [ ]:
def func(arg: String) = arg
// String => String
val arg = { str => func(str) }

In [ ]:
// String => String
// can we define similar strings to access rasters from S3?
def assetUri(key: String): String = s"https://geotrellis-workshop.s3.amazonaws.com/$key"
def bandUri(band: String): String = assetUri(s"landsat/LC81070352015218LGN00_$band.TIF")

bandUri("string")

In [ ]:
val greenBand = GeoTiffRasterSource(bandUri("B3"))
val redBand   = GeoTiffRasterSource(bandUri("B4"))
val nirBand   = GeoTiffRasterSource(bandUri("B5"))
val qaBand    = GeoTiffRasterSource(bandUri("BQA"))

In [ ]:
redBand.metadata
redBand.crs


## Read Overview

Landsat scenes in the `geotrellis-workshop` bucket have added overviews

- `RasterSource.resolutions`
- `RasterSource.resample`
- Reading tiles, `Option` return
- Rendering tiles
- Fixing `NODATA` value

In [ ]:
// 7751, 7891
// Overviews: 3876x3946, 1938x1973, 969x987, 485x494, 243x247
redBand.resolutions

In [ ]:
import geotrellis.raster.resample._
import geotrellis.raster.io.geotiff._

val overview = redBand.resample(
    resampleTarget = TargetCellSize(CellSize(500,500)), 
    method = NearestNeighbor, 
    strategy = Auto()
)


In [ ]:
// val opt: Option[String] = None

// opt.getOrElse("str")

In [ ]:
// Option
// Some(value)
// None

val result: Option[Raster[MultibandTile]] = overview.read()
val raster: Raster[MultibandTile] = result.get

val tile: Tile = raster.tile.band(0)

In [ ]:
Image(tile.renderPng().bytes)

In [ ]:
val histogram = tile.histogram
// histogram.values()

In [ ]:
val colorMap = ColorRamps.BlueToRed.toColorMap(histogram)

colorMap.colors

In [ ]:
// why the blue background?
Image(tile.withNoData(Some(0)).renderPng(colorMap).bytes)

## Read GeoJSON
GeoTrellis uses [circe](https://circe.github.io/circe/) library to parse and write JSON.

All we have to do is provide Encoders/Decoders for GIS types like [MultiPolygon](https://github.com/locationtech/geotrellis/blob/1a2ea84f7a15d790a13a75ede0fecee351ac4a7e/vector/src/main/scala/geotrellis/vector/io/json/GeometryFormats.scala#L157-L173)

In [ ]:
import _root_.io.circe._  
import _root_.io.circe.syntax._
import geotrellis.vector._

val json = scala.io.Source.fromURL(assetUri("gadm36/JPN_1_Chiba.geojson")).mkString
Text(json)

In [ ]:
json.parseJson

In [ ]:
import scala.util.Try

val chibaAoi = json.parseJson.as[MultiPolygon].right.get

// Option
// Some
// None

// Either
// Right(v: Int)
// Left(e: String)


In [ ]:
chibaAoi.asJson.spaces4

## Read Window from Landsat
- Using [Proj4J](https://github.com/locationtech/proj4j)

- [MultiPolygon Reproject ScalaDoc](https://geotrellis.github.io/scaladocs/latest/geotrellis/vector/reproject/Implicits$ReprojectMutliPolygon.html)
- [MultiPolygon Reproject Implicit Method](https://github.com/locationtech/geotrellis/blob/2f8348ac299d889282b7e6d379eed4696ece1dd7/vector/src/main/scala/geotrellis/vector/reproject/Implicits.scala#L89)                                   

In [ ]:
chibaAoi.extent

In [ ]:
redBand.crs

In [ ]:
redBand.read(chibaAoi.extent) // Oh no, we read None!

In [ ]:
import geotrellis.proj4._

val chibaAoiUtm = chibaAoi.reproject(LatLng, greenBand.crs)

In [ ]:
val chibaUtmExtent = chibaAoiUtm.extent
val chibaRedRaster = redBand.read(chibaUtmExtent).get

In [ ]:
val chibaRedBand = chibaRedRaster.tile.band(0).withNoData(Some(0))
Image(chibaRedBand.renderPng(colorMap).bytes)

## Rasterize AOI
Lets verify that AOI has been reprojected correctly by rasterizing it onto the Landsat scene

In [ ]:
val chibaMask: MutableArrayTile = chibaRedBand.mutable

chibaRedRaster.rasterExtent.foreach(chibaAoiUtm) { (x, y) =>
    chibaMask.set(x, y, Short.MaxValue)
}

// WARNING: this worked, but we just mutated the chibaRedBand Tile!
Image(chibaMask.renderPng(colorMap).bytes)

## Mask Clouds using QA Layer

In [ ]:
val qaTile: Tile = qaBand.read(chibaUtmExtent).get.tile.band(0).withNoData(Some(0))

In [ ]:
def maskClouds(tile: Tile): Tile =
  tile.combine(qaTile) { (v: Int, qa: Int) =>
    val isCloud = qa & 0x8000
    val isCirrus = qa & 0x2000
    if(isCloud > 0 || isCirrus > 0) { NODATA }
    else { v }
  }

In [ ]:
Image(maskClouds(chibaRedBand).renderPng(colorMap).bytes)

## Compute NDVI

In [ ]:
def ndvi (r: Double, ir: Double) : Double = {
    if (isData(r) && isData(ir)) {
        (ir - r) / (ir + r)
    } else {
      // https://github.com/locationtech/geotrellis/blob/master/raster/src/main/scala/geotrellis/raster/package.scala#L104-L111
      doubleNODATA
    }
}

In [ ]:
val chibaNirBand = nirBand.read(chibaUtmExtent).get.tile.band(0)

In [ ]:
// .convert(FloatConstantNoDataCellType)

// Landsat tiles are stored as Short (0 - 32767), NDVI should be Float (-1.0 .. 1.0)
val red = maskClouds(chibaRedBand).convert(FloatConstantNoDataCellType)
val nir = maskClouds(chibaNirBand).convert(FloatConstantNoDataCellType)

In [ ]:
val chibaNdvi = red.combineDouble(nir) { (r, ir) =>
    if (isData(r) && isData(ir)) {
        (ir - r) / (ir + r)
    } else {
      // https://github.com/locationtech/geotrellis/blob/master/raster/src/main/scala/geotrellis/raster/package.scala#L104-L111
      doubleNODATA
    }   
}

In [ ]:
val ndviColorMap = ColorMap.fromStringDouble(
    "0:ffffe5ff;0.1:f7fcb9ff;0.2:d9f0a3ff;0.3:addd8eff;0.4:78c679ff;0.5:41ab5dff;0.6:238443ff;0.7:006837ff;1:004529ff"
).get

Image(chibaNdvi.renderPng(ndviColorMap).bytes)

In [ ]:
val geotiff = GeoTiff(chibaNdvi, chibaRedRaster.extent, redBand.crs)
geotiff.write("ndvi.tif")

In [ ]:
Image(tile.withNoData(Some(0)).hillshade(CellSize(500, 500)).renderPng())